In [ ]:
#!pip install lazypredict
#!pip install -U sentence-transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from lazypredict.Supervised import LazyClassifier
import json

def write_json(data, path):
    """
            Write json file
    Args:
        param data
        param path to save json file
    Return:
        json file
    """
    output_dict = {
    'predictions': data.to_dict()
    }
    with open(path, "w", encoding="utf-8") as outfile:
        json.dump(output_dict, outfile, indent=4)


In [ ]:
root_drive_dir = '/content/drive/My Drive/Colab Notebooks/DataHub Projects/CrispyWork/edos/data/'

dataset = pd.read_csv(os.path.join(root_drive_dir, "edos_labelled_aggregated.csv"))
dataset = dataset[dataset['label_sexist'] == 'sexist'][["text", "label_category", "split"]]

#### train, dev, test split

In [ ]:
train = dataset[dataset['split'] == 'train']
train = train.reset_index(drop=True)
print('Train set shape:', train.shape)

dev = dataset[dataset['split'] == 'dev']
dev = dev.reset_index(drop=True)
print('Validation set shape:', dev.shape)

test = dataset[dataset['split'] == 'test']
test = test.reset_index(drop=True)
print('Test set shape:', test.shape)

Train set shape: (3398, 3)
Validation set shape: (486, 3)
Test set shape: (970, 3)


In [ ]:
train.head(2)

,text,label_category,split
0,[USER] Leg day is easy. Hot girls who wear min...,3. animosity,train
1,I get a new pussy every other week or whenever...,2. derogation,train


### load Sentence Transformer

In [ ]:
model = SentenceTransformer('cross-encoder/nli-deberta-base')

config.json:   0%|          | 0.00/975 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

In [ ]:
# Function to generate sentence embeddings
def generate_embeddings(dataframe):
    embeddings = []
    for sentence in dataframe['text']:
        embeddings.append(model.encode(sentence))
    return embeddings

# Generate embeddings for train, dev and test data
train_embeddings = generate_embeddings(train)
dev_embeddings = generate_embeddings(dev)
test_embeddings = generate_embeddings(test)


# Convert the lists of embeddings back to DataFrames
train_features = pd.DataFrame(train_embeddings)
dev_features = pd.DataFrame(dev_embeddings)
test_features = pd.DataFrame(test_embeddings)

train_labels = train['label_category']
dev_labels = dev['label_category']
test_labels = test['label_category']

#### model on dev set

In [ ]:
# LazyClassifier
clf = LazyClassifier()
models, dev_predictions = clf.fit(train_features, dev_features, train_labels, dev_labels)

 90%|████████▉ | 26/29 [03:30<00:14,  4.83s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036276 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 3398, number of used features: 768
[LightGBM] [Info] Start training from score -2.394370
[LightGBM] [Info] Start training from score -0.759453
[LightGBM] [Info] Start training from score -1.070466
[LightGBM] [Info] Start training from score -2.322800


100%|██████████| 29/29 [04:24<00:00,  9.12s/it]


In [ ]:
# save results in json file
json_file_path = "TaskB_deberta_lazyclassifier_dev.json"

write_json(dev_predictions, json_file_path)

#### model on test set

In [ ]:
# LazyClassifier
clf = LazyClassifier()
models, test_predictions = clf.fit(train_features, test_features, train_labels, test_labels)

 90%|████████▉ | 26/29 [03:19<00:14,  4.98s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035990 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 3398, number of used features: 768
[LightGBM] [Info] Start training from score -2.394370
[LightGBM] [Info] Start training from score -0.759453
[LightGBM] [Info] Start training from score -1.070466
[LightGBM] [Info] Start training from score -2.322800


100%|██████████| 29/29 [04:10<00:00,  8.63s/it]


In [ ]:
# save results in json file
json_file_path = "TaskB_deberta_lazyclassifier_test.json"

write_json(test_predictions, json_file_path)